In [297]:
from bs4 import BeautifulSoup 
import requests
import re
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

import string


from keras import optimizers

In [298]:
fullpath='https://en.wikipedia.org/wiki/List_of_nicknames_used_in_basketball'

In [299]:
soup=BeautifulSoup(requests.get(fullpath).text,'html.parser')

In [300]:
# print(soup)

In [301]:
# soup.findAll('div',{'class':'mw-content-ltr'})

In [302]:
uls=soup.findAll('ul')
# i=0
# for ul in uls:
#     print(i)
#     print(ul)
#     i+=1

In [303]:
namevecs=[]
for ul in uls:
    #print(i)
    #print(ul)
    lis=ul.findAll('li')
    namevec=[li.getText() for li in lis]
    namevecs+=namevec
   

In [304]:
nicknames=[]
for namevec in namevecs:
    #print(namevec)
    names=re.findall(r'\"(.*?)\"',namevec)
    nicknames+=names

In [305]:
len(nicknames)
x=nicknames[0]
x.find(' ')

3

In [327]:
#nicknamevec=[re.sub('^The ','',i) for i in nicknames]
nicknamevec=[re.sub('[^a-zA-Z .]+', ' ', i) for i in nicknames]
nicknamevec=[re.sub('  ', ' ', i) for i in nicknamevec]
nicknamevec=[(i.lower()+'*') for i in nicknamevec]
starters=[]

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

for i in nicknamevec:
    adder=i[0:5]
    if len(adder)==5:
        adder=re.sub('[^a-zA-Z .]+', ' ', adder) 
        starters.append(adder)
        inds=find(i,' ')
        for ind in inds:
            adder=i[ind+1:ind+6]
            if len(adder)==5:
                adder=re.sub('[^a-zA-Z .]+', ' ', adder) 
                starters.append(adder)
#nicknamevec
starters=set(starters)

print(len(starters))
print(starters)


682
{'lickf', 'gambl', 'dream', 'fella', 'might', 'arena', 'jewis', 'big o', 'smoot', 'white', 'snake', 'never', 'the c', 'mamba', 'matri', 'slama', 'refri', 'big p', 'bad n', 'agent', 'herma', 'doggi', 'boute', 'memo ', 'ray t', 'goose', 'purpl', 'vince', 'silas', 'whopp', 'blitz', 'midai', 'sofo ', 'thund', 'of ho', 'big c', 'o nea', 'fredd', 'shot ', 'broth', 'ready', 'cardi', 'midge', 'three', 'vanil', 'mutom', 'micro', 'curly', 'dance', 'sabas', 'houdi', 'round', 'contu', 'diggl', 'road ', 'my lo', 'foul ', 'beast', 'rock ', 'the j', 'easy ', 'icebe', 'mr. s', 'dirty', 'munst', 'zeke ', 'roost', 'duran', 'the h', 'sam i', 'serge', 'chief', 'truck', 'cream', 'berli', 'd fis', 'count', 'k mar', 'wilt ', 'maple', 'o san', 'mites', 'buddy', 'tree ', 'man h', 'moose', 'run t', 'life ', 'paper', 'junky', 'shaqu', 'kicks', 'googs', 'zen m', 'speci', 'fab f', 'wave ', 'attac', 'trick', 'big m', 'swagg', 'hoodi', 'game ', 'coke ', 'force', 'sir s', 'mt. m', 'blind', 'green', 'nervo', 'brow

In [307]:
chars=[y for x in nicknamevec for y in x]
chars=sorted(list(set(chars)))
n_vocab=len(chars)
chars
char_to_int=dict((c,i) for i,c in enumerate(chars))
char_to_int

{' ': 0,
 '*': 1,
 '.': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28}

In [308]:
seq_length=5
dataX=[]
dataY=[]
for j in nicknamevec:
    n_chars=len(j)
    for i in range(0, n_chars - seq_length, 1):
        seq_in = j[i:i + seq_length]
        seq_out = j[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
    n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  4008


In [309]:
X=np.reshape(dataX,(n_patterns,seq_length,1))
X=X/float(n_vocab)
y=np_utils.to_categorical(dataY)

In [310]:
X

array([[[0.75862069],
        [0.34482759],
        [0.24137931],
        [0.        ],
        [0.17241379]],

       [[0.34482759],
        [0.24137931],
        [0.        ],
        [0.17241379],
        [0.10344828]],

       [[0.24137931],
        [0.        ],
        [0.17241379],
        [0.10344828],
        [0.62068966]],

       ...,

       [[0.24137931],
        [0.        ],
        [0.13793103],
        [0.5862069 ],
        [0.5862069 ]],

       [[0.        ],
        [0.13793103],
        [0.5862069 ],
        [0.5862069 ],
        [0.75862069]],

       [[0.13793103],
        [0.5862069 ],
        [0.5862069 ],
        [0.75862069],
        [0.34482759]]])

In [311]:
cells=256
dropout=0.3
hidden_layers=5
#lr=0.1
opti=optimizers.adam(clipnorm =1)

In [312]:
model=Sequential()
model.add(LSTM(cells,input_shape=(X.shape[1],X.shape[2]),return_sequences=True))
model.add(Dropout(dropout))
for i in range(hidden_layers-1):
    model.add(LSTM(cells,return_sequences=True))
    model.add(Dropout(dropout))
model.add(LSTM(cells))
model.add(Dropout(dropout))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=opti)

In [313]:
#filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]

In [314]:
model.fit(X,y,epochs=100,batch_size=64)

Epoch 1/100
4008/4008 [==============================] - 42s 11ms/step - loss: 3.0668
Epoch 2/100
4008/4008 [==============================] - 33s 8ms/step - loss: 2.9907
Epoch 3/100
4008/4008 [==============================] - 29s 7ms/step - loss: 2.9914
Epoch 4/100
4008/4008 [==============================] - 30s 7ms/step - loss: 2.9882
Epoch 5/100
4008/4008 [==============================] - 30s 7ms/step - loss: 2.9823
Epoch 6/100
4008/4008 [==============================] - 30s 8ms/step - loss: 2.9815
Epoch 7/100
4008/4008 [==============================] - 29s 7ms/step - loss: 2.9782
Epoch 8/100
4008/4008 [==============================] - 30s 7ms/step - loss: 2.9785
Epoch 9/100
4008/4008 [==============================] - 29s 7ms/step - loss: 2.9702
Epoch 10/100
4008/4008 [==============================] - 31s 8ms/step - loss: 2.9695
Epoch 11/100
4008/4008 [==============================] - 30s 7ms/step - loss: 2.9548
Epoch 12/100
4008/4008 [==============================] - 30s 

4008/4008 [==============================] - 29s 7ms/step - loss: 0.8606
Epoch 97/100
4008/4008 [==============================] - 29s 7ms/step - loss: 0.8482
Epoch 98/100
4008/4008 [==============================] - 29s 7ms/step - loss: 0.8236
Epoch 99/100
4008/4008 [==============================] - 29s 7ms/step - loss: 0.8643
Epoch 100/100
4008/4008 [==============================] - 28s 7ms/step - loss: 0.8123


In [315]:
int_to_char=dict((i,c) for i,c in enumerate(chars))

In [318]:
start=np.random.randint(0,len(dataX)-1)
pattern=dataX[start]
nameout='peter'
pattern=[char_to_int[char] for char in nameout]
patternin=pattern
print(pattern)

[4, 14, 3, 5, 13]


In [319]:
for i in range(25):
    x=np.reshape(pattern,(1,len(pattern),1))
    x=x/float(n_vocab)
    prediction=model.predict(x,verbose=0)
    index=np.argmax(prediction)
    result=int_to_char[index]
    nameout+=result
    seq_in=[int_to_char[value] for value in pattern]
    #print(result)
    pattern.append(index)
    pattern=pattern[1:len(pattern)]
    if result=='*':
        break

In [320]:
print(nameout)

black mam*


In [321]:
thes=[re.findall('^The',nickname) for nickname in nicknames]
thes

[['The'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 ['The'],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 ['The'],
 ['The'],
 ['The'],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['The'],
 ['The'],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 ['The'],
 [],
 ['The'],
 [],
 ['The'],
 ['The'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 ['The'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['The'],
 [],
 ['The'],
 ['The'],
 [],
 ['The'],
 [],
 [],
 ['The'],
 [],
 ['The'],
 [],
 [],
 ['The'],
 [],
 [],


In [322]:
lets=string.ascii_lowercase
invec=[]
starters=list(starters)
for l in lets:
    starters.append('the '+l)
    starters.append('big '+l)
    starters.append('all '+l)
    starters.append('any '+l)
    starters.append('old '+l)
    starters.append('mr. '+l)
    starters.append('dr. '+l)
    
starters=list(set(starters))

In [335]:
len(starters)

682

In [336]:
outnames=[]
for instart in starters:
    nameout=instart
    pattern=[char_to_int[char] for char in nameout]
    for i in range(25):
        x=np.reshape(pattern,(1,len(pattern),1))
        x=x/float(n_vocab)
        prediction=model.predict(x,verbose=0)
        index=np.argmax(prediction)
        result=int_to_char[index]
        nameout+=result
        seq_in=[int_to_char[value] for value in pattern]
        #print(result)
        pattern.append(index)
        pattern=pattern[1:len(pattern)]
        if result=='*':
            print(nameout)
            outnames.append(nameout)
            break
        

lickface*
gambler*
dream*
fella*
might film*
arena*
jewish jordan*
big oumbers*
smoote*
white hopse*
snake*
never met a shot rod*
the canceaie*
mamba*
matrix*
slama jama*
refrigerator*
big p*
bad news*
agent zeam of all time*
herman riverwalk*
doggie*
boute*
memo lordan*
ray the latrr*
goose*
purpless*
vince*
silas*
whopper*
blitz kids*
midair*
sofo mase*
thunder*
of horse*
big cordially*
o nealovic*
freddie*
shot rod*
brothers*
ready*
cardiac kemba*
midget*
three*
vanilla gorille*
mutom the latrr*
microwy cigar*
curly*
dance insse*
sabas jr.*
houdini of test*
round of west*
contusioe*
diggler*
road warrin*
my lou*
beast of sesvice pervis*
rock a*
the jatk*
easy ed*
iceberg slim*
mr. silas*
dirty kurt*
munster*
zeke dunkin dutchman*
rooster*
durantly*
the hardwood*
sam i am*
serge process*
chief*
truck*
cream of all time*
berlinator*
d fish*
country*
k marty*
wilt chamberneezy*
maple j*
o san*
mites*
buddy love*
tree tarrin*
man highlight film*
moose*
run tmc*
life orocess*
paper dance

In [329]:
1+1


2

In [330]:
cells=256
dropout=0.3
hidden_layers=5
#lr=0.1
opti=optimizers.adam(clipnorm =1)

In [331]:
model50=Sequential()
model50.add(LSTM(cells,input_shape=(X.shape[1],X.shape[2]),return_sequences=True))
model50.add(Dropout(dropout))
for i in range(hidden_layers-1):
    model50.add(LSTM(cells,return_sequences=True))
    model50.add(Dropout(dropout))
model50.add(LSTM(cells))
model50.add(Dropout(dropout))
model50.add(Dense(y.shape[1],activation='softmax'))
model50.compile(loss='categorical_crossentropy',optimizer=opti)

In [332]:
model50.fit(X,y,epochs=50,batch_size=64)

Epoch 1/50
4008/4008 [==============================] - 24s 6ms/step - loss: 3.0735
Epoch 2/50
4008/4008 [==============================] - 17s 4ms/step - loss: 2.9893
Epoch 3/50
4008/4008 [==============================] - 18s 4ms/step - loss: 2.9854
Epoch 4/50
4008/4008 [==============================] - 18s 5ms/step - loss: 2.9851
Epoch 5/50
4008/4008 [==============================] - 19s 5ms/step - loss: 2.9803
Epoch 6/50
4008/4008 [==============================] - 20s 5ms/step - loss: 2.9829
Epoch 7/50
4008/4008 [==============================] - 21s 5ms/step - loss: 2.9789
Epoch 8/50
4008/4008 [==============================] - 24s 6ms/step - loss: 2.9744
Epoch 9/50
4008/4008 [==============================] - 27s 7ms/step - loss: 2.9775
Epoch 10/50
4008/4008 [==============================] - 34s 8ms/step - loss: 2.9690
Epoch 11/50
4008/4008 [==============================] - 25s 6ms/step - loss: 2.9535
Epoch 12/50
4008/4008 [==============================] - 26s 6ms/step - lo

In [337]:
for instart in starters:
    nameout=instart
    pattern=[char_to_int[char] for char in nameout]
    for i in range(25):
        x=np.reshape(pattern,(1,len(pattern),1))
        x=x/float(n_vocab)
        prediction=model50.predict(x,verbose=0)
        index=np.argmax(prediction)
        result=int_to_char[index]
        nameout+=result
        seq_in=[int_to_char[value] for value in pattern]
        #print(result)
        pattern.append(index)
        pattern=pattern[1:len(pattern)]
        if result=='*':
            print(nameout)
            outnames.append(nameout)
            break
        

lickft*
gambler*
dream*
fella*
might faced asssovers*
arena*
jewiss*
big oate*
smoot*
white coothers*
snake amack magic*
never magic*
the cig three*
mamba*
matrin*
slaman*
refrign towers*
big pare*
bad newsic*
agent zersicr boa*
herman*
doggie*
bouter*
memo bocker*
ray tay*
goose*
purple oan*
vince*
silas*
whopp*
blitz mid*
midair*
sofo towers*
thundery*
of hordan*
big cer*
o neal*
freddne*
shot touer*
brothers*
ready*
cardiac kew*
midget*
three*
vaniler*
mutom three*
microoy*
curly*
dance insdan*
sabas*
houdinan*
round*
contu cigar*
diggler*
road wam*
my lor*
foul she latric pan*
beast*
rock b*
the jares*
easy of servic*
iceberg sowers*
mr. silas*
dirty*
munst*
zeke dream*
roost*
durant*
the hammer*
sam iighlight faced asssovers*
serge prince*
chief*
truck*
cream*
berlin towers*
d fish*
count*
k mar*
wilt touer*
maple james*
o san*
mites*
buddy lou*
tree tarty*
man highlight faced asssovers*
moose*
run the latric pan*
life pervic*
paper magic*
junky*
shaqu mf servic*
kicks bocker*
goo

In [338]:
len(nicknames)

615

In [339]:
len(outnames)

1345

In [340]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 5, 256)            264192    
_________________________________________________________________
dropout_25 (Dropout)         (None, 5, 256)            0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 5, 256)            525312    
_________________________________________________________________
dropout_26 (Dropout)         (None, 5, 256)            0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 5, 256)            525312    
_________________________________________________________________
dropout_27 (Dropout)         (None, 5, 256)            0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 5, 256)            525312    
__________